<a href="https://colab.research.google.com/github/francisan17/CuPractice/blob/main/Cuterm1x1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from ase import Atoms
from mace.calculators import mace_mp
from ase.build import surface
from ase.constraints import FixAtoms, UnitCellFilter
from ase.optimize import QuasiNewton
from ase.visualize import view
from ase.io import read, write
from ase.optimize import LBFGS
import numpy as np
import pandas as pd
from tabulate import tabulate
from cu2o_bulk_c import cu2o_bulk, cu2o100, Cuterm1x1


bulk = cu2o_bulk()
bulk.calc = mace_mp(model="large", dispersion=True, default_dtype="float64", device='cuda')
ucf = UnitCellFilter(bulk)
LBFGS(ucf).run(fmax=0.01)
E_bulk = bulk.get_potential_energy()
bulk_oxygens = bulk[bulk.symbols=='O']
print(f"Number of Oxygen atoms in bulk: {len(bulk_oxygens)}")
#write('bulk.xyz', bulk)

n_layers = { 3,4,5,6,7,8,9,10}
Cu2Otable={}

n_layers_list=[]
slab_oxygens_list=[]
E_surf_list=[]


for n_layers in range(3,11):
  vacuum=10
  slab = Cuterm1x1(bulk, n_layers, vacuum)
  #write('slab.xyz', slab)
  slab_oxygens = slab[slab.symbols=='O']
  Cu2Otable[n_layers]=len(slab_oxygens)
  print(f"Number of Oxygen atoms in slab: {len(slab_oxygens)}")

  slab.calc = mace_mp(model="large", dispersion=True, default_dtype="float64", device='cuda')
  E_slab = slab.get_potential_energy()
  E_cleav = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])
  print(f'{n_layers=} {E_cleav=}')

  qn = LBFGS(slab, trajectory='Cuterm1x1.traj')
  qn.run(fmax=0.01)
  E_slab = slab.get_potential_energy()

  ##Calc Surface Energy
  E_surf = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])
  print(f'{n_layers=} {E_surf=}')

  n_layers_list.append(n_layers)
  slab_oxygens_list.append(len(slab_oxygens))
  E_surf_list.append(E_surf)

df = pd.DataFrame({'Number of Layers': n_layers_list, 'Number of Oxygen Atoms':slab_oxygens_list, 'Surface Energy': E_surf_list})
table=tabulate(df, headers = 'keys', tablefmt = 'fancy_grid')
df.to_csv('Cuterm1x1.csv', index=False)
print(table)


Cached MACE model to /root/.cache/mace/5f5yavf3
Using Materials Project MACE for MACECalculator with /root/.cache/mace/5f5yavf3
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
Using TorchDFTD3Calculator for D3 dispersion corrections (see https://github.com/pfnet-research/torch-dftd)


<ipython-input-4-b9ff6de182f2>:17: FutureWarning: Import UnitCellFilter from ase.filters
  ucf = UnitCellFilter(bulk)
/usr/local/lib/python3.10/dist-packages/torch_dftd/torch_dftd3_calculator.py:98: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  cell: Optional[Tensor] = torch.tensor(


       Step     Time          Energy          fmax
LBFGS:    0 12:40:40      -30.685465        0.329906
LBFGS:    1 12:40:41      -30.690012        0.313324
LBFGS:    2 12:40:43      -30.727313        0.056428
LBFGS:    3 12:40:43      -30.728281        0.007799
Number of Oxygen atoms in bulk: 2
Number of Oxygen atoms in slab: 6
Using Materials Project MACE for MACECalculator with /root/.cache/mace/5f5yavf3
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
Using TorchDFTD3Calculator for D3 dispersion corrections (see https://github.com/pfnet-research/torch-dftd)
n_layers=3 E_cleav=0.11138727898774493
       Step     Time          Energy          fmax
LBFGS:    0 12:40:43      -88.212283        1.767347
LBFGS:    1 12:40:46      -88.368750        1.396075
LBFGS:    2 12:40:46      -88.548865        0.896234
LBFGS:    3 12:40:46      -88.611736        0.822606
LBFGS:    4 12:40:49      -88.773803        1.003785
LBFGS:    5 12:40:

In [2]:
!pip install mace-torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.7/387.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 28.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 25.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-

In [3]:
!pip install torch-dftd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.4/681.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 779.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.2/809.2 kB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 51.5 MB/s eta 0:00:00
  Attempting uninstall: